<a href="https://colab.research.google.com/github/Flipeviotto/a_little_is_enough_attack/blob/main/Ataque_LIE_Version5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [366]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Subset, DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor, Normalize, Compose
from sklearn.metrics import precision_recall_fscore_support

In [367]:
maior = 0


# Classe Arguments para encapsular hiperparâmetros
class Arguments:
    def __init__(self):
        self.train_batch_size = 83
        self.test_batch_size = 83
        self.epochs = 150
        self.lr = 0.1                 # taxa de aprendizado, controla o tamanho da atualização dos parametros
        self.momentum = 0.9           # cria um termo de velocidade que influencia a atualização dos parametros
        self.l2 = 1e-4                # parametro de decaimento de peso, penalisa pesos muito grantes no treinamento
        self.n_workers = 51           # quantidade de dispositivos na simulação
        self.n_corrupted_workers = 12 # quantidade de dispositivos maliciosos
        self.no_cuda = False

args = Arguments()
use_cuda = not args.no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# A rede

In [368]:
class MLP(nn.Module):
    def __init__(self):
      super(MLP, self).__init__()
      self._f1 = torch.nn.Linear(28 * 28, 100)
      self._f2 = torch.nn.Linear(100, 10)

    def forward(self, x):
      x = torch.nn.functional.relu(self._f1(x.view(-1, 28 * 28)))
      x = torch.nn.functional.log_softmax(self._f2(x), dim=1)
      return x

# preparar dados

In [369]:
def getdata():
    return MNIST(root="./data", train=True, download=True, transform=ToTensor()), MNIST(root="./data", train=False, download=True, transform=ToTensor())

def prepare_mnist_dataset():
    train_dataset, test_dataset = getdata()  # Obtém os datasets de treinamento e teste

    # Divisão do dataset de treinamento em partes para cada worker
    train_sets = []
    parte_size = len(train_dataset) // args.n_workers

    for i in range(args.n_workers):
        inicio = i * parte_size
        fim = (i + 1) * parte_size
        parte = Subset(train_dataset, range(inicio, fim))
        train_sets.append(parte)

    # Lidando com sobras de dados
    if len(train_dataset) % args.n_workers != 0:
        sobras = Subset(train_dataset, range(args.n_workers * parte_size, len(train_dataset)))
        train_sets.append(sobras)

    # Criação dos DataLoaders
    trainloaders = []
    for train_set in train_sets:
        trainloaders.append(DataLoader(train_set, batch_size=args.train_batch_size, shuffle=True))

    testloader = DataLoader(test_dataset, batch_size=args.test_batch_size, shuffle=False)

    return trainloaders, testloader

# treino

In [370]:
def train(args: Arguments, models: list, device, train_loaders, optimizers, criterion, epoch):

    for j in range(args.n_workers):
        models[j].train()
        for data, target in train_loaders[j]:
            data, target = data.to(device), target.to(device)
            output = models[j](data)
            loss = criterion(output, target)
            optimizers[j].zero_grad()
            loss.backward()

            # Clipping para evitar explosão de gradientes
            torch.nn.utils.clip_grad_norm_(models[j].parameters(), max_norm=1.0)
            optimizers[j].step()

# agregação

In [371]:
def fedavg_aggregation(models: list, global_model):
    all_params = [list(model.parameters()) for model in models]
    average_params = [torch.stack(params).mean(0) for params in zip(*all_params)]

    for global_param, avg_param in zip(global_model.parameters(), average_params):
        global_param.data.copy_(avg_param)

    # Atualiza os modelos locais com os pesos do modelo global
    for model in models:
        model.load_state_dict(global_model.state_dict())

# teste

In [372]:
def test(global_model, device, test_loader, epoch):
    global maior

    global_model.eval()
    all_predictions = []
    all_targets = []
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = global_model(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            all_predictions.extend(pred.cpu().numpy().flatten())
            all_targets.extend(target.cpu().numpy().flatten())

    total_samples = len(test_loader.dataset)
    precision, recall, f1_score, _ = precision_recall_fscore_support(all_targets, all_predictions, average='weighted', zero_division=1)
    accuracy = (correct / total_samples) * 100
    #if(accuracy>maior):
      #maior = accuracy
      #print(f"Maior acurácia: {maior}")
    with open(f"teste{wm}.txt", 'a') as file:
      file.write(f'epoca {epoch} | Accuracy: {accuracy:.2f}% | Correct: {correct}/{total_samples} | Precision: {precision:.4f} | Recall: {recall:.4f} | F1-score: {f1_score:.4f}\n')
    print(f'Teste: Precisão: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1_score:.4f}, Acurácia: {accuracy:.2f}%')


# Ataque "a little is enough"

Ainda em construção

In [373]:
from scipy.stats import norm

def ataque(args:Arguments, model:list, workers_maliciosos:list):
  """
    Altera os parametros dos workers maliciosos.

    input
    args: contem informações sobre o modelo
    model: lista de modelos
    workers_maliciosos: lista de indices dos workers maliciosos

    output
    nenhum
  """

  p = 0

  #calcular a taxa de perturbação
  s = args.n_workers/2 + 1 - len(workers_maliciosos)
  possibilidade = (args.n_workers-s)/args.n_workers
  z = norm.ppf(possibilidade)
  #print(f"taxa de perturbação real: {z}")



  # instancia modelos auxiliares
  soma_parametros = MLP().to(device)
  media = MLP().to(device)
  desvio_padrao = MLP().to(device)
  desvio_padrao_aux = MLP().to(device)


  #print("soma inicio: ")
  #for param in soma_parametros.parameters():
  #  print(param)
  #  break;


  # inicializa modelos com zero
  for soma, param_media, desvio_aux, desvio_pad in zip(soma_parametros.parameters(), media.parameters(), desvio_padrao_aux.parameters(), desvio_padrao.parameters()):
    soma.data.zero_()
    desvio_aux.data.zero_()


  #print("soma zerada:")    # para saber se esta zerando
  #for param in soma_parametros.parameters():
  #  print(param)
  #  break;


  # somar parametros
  with torch.no_grad():
    for malicioso in workers_maliciosos:
      #print(f"Pesos do modelo para o worker {malicioso}:")
      #for name, param in model[malicioso].named_parameters():
      #  print(f"{name}: {param.data}")
      #  break
      #print("\n")


      for modelo_param, soma_param in zip(model[malicioso].parameters(), soma_parametros.parameters()):
        soma_param.data += modelo_param.data


    #print("soma final: ")
    #for param in soma_parametros.parameters():
    #  print(param)
    #  break;


    #calcular media
    for soma_param, parametro_media in zip(soma_parametros.parameters(), media.parameters()):
      parametro_media.data = soma_param.data/len(workers_maliciosos)
      #if p == 0:
      #  print("media: ")
      #  print(parametro_media.data)
      #  p = 1
    #p=0

    # calculo desvio padrao
    for malicioso in workers_maliciosos:
      #p=0
      for parametro_modelo, variancia, parametro_media in zip(model[malicioso].parameters(), desvio_padrao_aux.parameters(), media.parameters()):
        variancia.data += ((parametro_modelo.data - parametro_media.data) ** 2)
        #if p==0:
        #  print("worker: ")
        #  print(parametro_modelo.data)
        #  p=1

    for variancia, desvio in zip(desvio_padrao_aux.parameters(), desvio_padrao.parameters()):
      desvio.data = torch.sqrt(variancia.data / len(workers_maliciosos))

    # alterar parametros
    for malicioso in workers_maliciosos:
      for parametro, desvio, parametro_media in zip(model[malicioso].parameters(), desvio_padrao.parameters(), media.parameters()):
        parametro.data = parametro_media.data + z * desvio.data
        #if(p==1):
        #  print("desvio")
        #  print(desvio.data)
        #  p=0

  soma_parametros = None
  media = None
  desvio_padrao = None
  desvio_padrao_aux = None

  return 0


# main
faz o gerenciamento do treino, ataque e teste.

In [374]:
def main(wm):                                                                       # recebe a quantidade de workes maliciosos
  models = [MLP().to(device) for _ in range(args.n_workers)]                      # cria uma lista de workers
  optimizers = [optim.SGD(model.parameters(), lr= args.lr, momentum=args.momentum, weight_decay= args.l2) for model in models]  # SGD com momentum e L2 regularização (weight_decay)
  criterion = nn.CrossEntropyLoss()

  workers_maliciosos = []           # cria lista para armazenar os workes maliciosos
  #workers_maliciosos = random.sample(range(args.n_workers), args.n_corrupted_workers)
  for i in range(wm):               # marca os numeros dos workers maliciosos
    workers_maliciosos.append(i)

  # Criando os loaders de dados para cada trabalhador e o loader de teste global
  trainloaders, testloader = prepare_mnist_dataset()
  global_model = MLP().to(device)

  # Loop de treinamento federado
  for epoch in range(1, args.epochs + 1):
    train(args, models, device, trainloaders, optimizers, criterion, epoch)

    if epoch % 1 == 0:
      if wm>1:
        ataque(args, models, workers_maliciosos)
      fedavg_aggregation(models, global_model)
      test(global_model, device, testloader, epoch)

In [ ]:
if __name__ == "__main__":
  for wm in [args.n_corrupted_workers]:                  # passo com as quantidades de workers maliciosos em cada simulação
    print(f"Quantidade de workers maliciosos: {wm}")
    with open(f"teste{wm}.txt", 'w') as file:
      file.write(f"Quantidade de workers maliciosos: {wm}\n")
    main(wm)
    print(f"Maior acurácia: {maior}")
    with open(f"teste51.txt", 'a') as file:
      file.write(f"Maior acurácia: {maior}\n")